In [277]:
from faker import Faker
import pandas as pd
import random as rd

SEED = 29
Faker.seed(SEED)
rd.seed(SEED)

fk = Faker('pt-BR')
QNT = 10

In [278]:
def generate_addresses():
    complements = ['Casa 2', 'Apto. 162', 'Casa B', 'Galpão A', None, None, None, None, None, None]
    raw_addresses = []

    for ad in [fk.address() for _ in range(QNT)]:
        split_ad = ad.split('\n')
        street_address = split_ad[0].split(',')
        street = street_address[0]
        number = street_address[1] if len(street_address) > 1 else 0
        temp, state = split_ad[2].split('/')
        postal_code, city = temp.split(' ', maxsplit=1)
        complement = rd.choice(complements)

        raw_addresses.append(('Brasil', state, city, street, number, postal_code, complement))

    return raw_addresses

In [279]:
addresses = pd.DataFrame(
    data=generate_addresses(),
    columns=['country', 'state', 'city', 'street', 'number', 'postal_code', 'complement']
)

display(addresses)

,country,state,city,street,number,postal_code,complement
0,Brasil,MT,Monteiro,Trevo de da Paz,9,66001-735,None
1,Brasil,SE,Jesus,Estrada de Machado,73,98386-448,Apto. 162
2,Brasil,RS,Leão,Lagoa Sá,89,52425186,None
3,Brasil,AL,Silveira,Vale Leonardo Duarte,14,66826240,None
4,Brasil,SC,Alves,Vereda Moreira,0,08022753,None
5,Brasil,AM,Pinto,Alameda José Correia,3,63338353,None
6,Brasil,RO,Monteiro do Sul,Viela Leão,6,92461-606,Apto. 162
7,Brasil,PE,Cavalcanti de Ramos,Distrito Cavalcante,89,53707393,None
8,Brasil,RN,Sampaio,Área Porto,90,40143-531,None
9,Brasil,MA,Gonçalves dos Dourados,Colônia da Cunha,15,33634-485,None
